# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [1]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [2]:
def try_generate_keys(nlen = 2048):
    p = cu.getPrime(nlen // 2)
    q = cu.getPrime(nlen // 2)
    n = p*q

    f = (p-1)*(q-1)

    e_min = 65537       # inclusive
    e_max = pow(2, 256) # exclusive
    e = random.randint(e_min, min(e_max - 1, f))
    while math.gcd(f, e) != 1:
        e += 1
        if e >= e_max or e > f:
            e = e_min

    d = cu.inverse(e, f)
    return d, e, n, p, q, f

def verify_keys(d, e, n, p, q, f, nlen = 2048):
    if not (65537 <= e and e < pow(2, 256)): return False
    if not (math.lcm(p-1, q-1) >= e * pow(2, nlen // 2)): return False
    if not (pow(2, (nlen-1) // 2) < p and p < pow(2, nlen // 2)): return False
    if not (pow(2, (nlen-1) // 2) < q and q < pow(2, nlen // 2)): return False
    if not (abs(p-q) > pow(2, (nlen // 2) - 100)): return False
    return True

def generate_keys(nlen = 2048):
    d, e, n = None, None, None
    i = 0
    success = False
    while not success:
        i += 1
        print(f'Trying to generate keys for the {i}. time...')
        d, e, n, p, q, f = try_generate_keys(nlen)
        success = verify_keys(d, e, n, p, q, f, nlen)
    print('Key generation successful!')
    print('Size of n in bits:', cu.size(n))

    return d, e, n

In [3]:
d, e, n = generate_keys()

Trying to generate keys for the 1. time...
Key generation successful!
Size of n in bits: 2048


## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [4]:
def encipher(message, e, n):
    M = cu.bytes_to_long(bytes(message, 'utf-8'))
    C = pow(M, e, n)
    return C

def decipher(C, d, n):
    M = pow(C, d, n)
    print(M)
    return str(cu.long_to_bytes(M), 'utf-8')

In [5]:
# Test
message = "Ala ma kota"

C = encipher(message, e, n)
print('Enciphered:', C)
M = decipher(C, d, n)
print('Deciphered:', M)

Enciphered: 15356173062884157861339793399266286756905978285695269897089610486279767871052855245868217808117797364317917910371379446925085842372056958069320431678727802447029962390334408496032511251247817049179892508563213389924791141442626049317150819832069929274865505655895512092374766905204076795190897032435953787206931066519355206879712070050157379612086280668392370468643249712689206777824872097551019390839667862268899536598297486706348527758233033024198823141161588606157106057590121907602463124435392791539772691495534386944999982609194670609981087576214956820251021283451644399474300014825102667391857968767739725658664
79091985525906541409956961
Deciphered: Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [6]:
def sign(M, d, n, H=hl.sha256):
    b = bytes(message, 'utf-8')
    h = int(H(b).hexdigest(), 16)
    SIG = pow(h, d, n)
    return (M, SIG)

def verify(M, SIG, e, n, H=hl.sha256):
    M = bytes(message, 'utf-8')
    h = int(H(M).hexdigest(), 16)
    VER = pow(SIG, e, n)
    return VER == h


In [7]:
M = 'Ala ma kota'

M, SIG = sign(M, d, n)
print('Message  :', M)
print('Signature:', SIG)
verified = verify(M, SIG, e, n)
print('Verified :', verified)

Message  : Ala ma kota
Signature: 11204643288673201383080901719991630610102758921137817420075761070172747994839581295564314569542000622685520008777648169346503878463766320383101466846476671554491751781680625495333620027557563429781999875363901051868704876003311424809122045953988220878765975763961033798949638869071262625838932649770055320379128275802009197887549236769529499873404929190327924611843095186941702382085608410475406160207523518342935063715263120365001674741960728657931610568942641949256310588342956061318132589834984482701492051460995505500509699770078153342929032565664602485396542640767224703969646100467578554546137727830966286040695
Verified : True


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu. (n to długość skrótu)

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [8]:
def bit_concat(a, b, len):
    return a << len | b

In [9]:
def mgf1(M, len, hlen=32, H=hl.sha256): 
    M_num = cu.bytes_to_long(M)
    M_num_size = cu.size(M_num)
    output = 0
    actual_length = 0
    for i in range(math.ceil(len / hlen)):
        input = bit_concat(M_num, i, 32)
        input_bytes = input.to_bytes(math.ceil(M_num_size / 8) + 4, byteorder='big')

        tmp = H(input_bytes);
        tmp_int = int(tmp.hexdigest(), 16)

        actual_length += hlen
        output = bit_concat(output, tmp_int, hlen * 8)

    return output >> ((actual_length - len) * 8)



In [10]:
# Test

# Data from wikipedia:
# b"bar", 50, sha256
# 382576a7841021cc28fc4c0948753fb8312090cea942ea4c4e735d10dc724b155f9f6069f289d61daca0cb814502ef04eae1

mask = mgf1(b'bar', 50)
expected_mask = '0x382576a7841021cc28fc4c0948753fb8312090cea942ea4c4e735d10dc724b155f9f6069f289d61daca0cb814502ef04eae1'
print('Expected:', expected_mask)
print('Actual  :', hex(mask))
print('Equal?  :', hex(mask) == expected_mask)

Expected: 0x382576a7841021cc28fc4c0948753fb8312090cea942ea4c4e735d10dc724b155f9f6069f289d61daca0cb814502ef04eae1
Actual  : 0x382576a7841021cc28fc4c0948753fb8312090cea942ea4c4e735d10dc724b155f9f6069f289d61daca0cb814502ef04eae1
Equal?  : True


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [11]:
# Utils

def generate_bit_mask(bit_len, bit_offset):
    # see generate_byte_mask for explanation
    return ((1 << bit_len) - 1) << bit_offset

def generate_byte_mask(byte_len, byte_offset):
    # returns 0xFFFFFF..FF0000..00, with a length of byte_len bytes and byte_offset bytes, for the 1s and 0s part respctively
    return generate_bit_mask(byte_len*8, byte_offset*8)

def nullify_first_n_bits(value, n, vlen):
    # sets first n bits (most significant) to 0
    return value & generate_bit_mask(vlen-n, 0)

def extract_bytes(value, len, right_offset):
    # extracts the bytes from value (skips 'right_offset' starting from the least significant bytes, and returns 'length' next bytes)
    return (value & generate_byte_mask(len, right_offset)) >> (right_offset*8)


In [12]:
def oaep_encrypt(M, k=256, hlen=32, H=hl.sha256):
    M_num = cu.bytes_to_long(bytes(M, 'utf-8'))
    mlen = math.ceil(cu.size(M_num) / 8) 
    if mlen > k - 2*hlen - 2:
        raise RuntimeError('Message too big')
    
    lHash = int(H(bytes("", 'utf-8')).hexdigest(), 16) 
    PS = 0
    # We don't need to make PS a certain length, since it's just the number 0
    # for _ in range(k-mlen-2*hlen-2):
    #     PS <<= 8
    
    DB = bit_concat(lHash, PS, (k-mlen-2*hlen-2) * 8)
    DB = bit_concat(DB, 0x01, 8)
    DB = bit_concat(DB, M_num, mlen * 8)

    seed = cu.getRandomNBitInteger(hlen * 8)

    dbMask = mgf1(cu.long_to_bytes(seed), k-hlen-1)
    maskedDB = DB ^ dbMask

    seedMask = mgf1(cu.long_to_bytes(maskedDB), hlen)
    maskedSeed = seed ^ seedMask

    EM = bit_concat(maskedSeed, maskedDB, (k-hlen-1) * 8) # not concating 0x00 at front, since it's invisible to python
    return EM, mlen

def oaep_decrypt(EM, mlen, k=256, hlen=32, H=hl.sha256):
    maskedDB = extract_bytes(EM, k-hlen-1, 0)
    maskedSeed = extract_bytes(EM, hlen, k-hlen-1)
    Y = extract_bytes(EM, 1, k-1)

    lHash = int(H(bytes("", 'utf-8')).hexdigest(), 16) 
    seedMask = mgf1(cu.long_to_bytes(maskedDB), hlen)
    seed = maskedSeed ^ seedMask

    dbMask = mgf1(cu.long_to_bytes(seed), k-hlen-1)
    DB = maskedDB ^ dbMask

    M = extract_bytes(DB, mlen, 0)
    one = extract_bytes(DB, 1, mlen)
    PS = extract_bytes(DB, k-mlen-2*hlen-2, mlen + 1)
    lHash2 = extract_bytes(DB, hlen, k-2*hlen-1)

    if Y != 0x00 and one != 0x01 and lHash != lHash2:
        raise RuntimeError('Validation failed')

    return str(cu.long_to_bytes(M), 'utf-8')


In [13]:
M = "Ala ma kota"
print(M)

EM, mlen = oaep_encrypt(M)
print(hex(EM))

DM = oaep_decrypt(EM, mlen)

print(DM)

Ala ma kota
0x18267887881fc9ff486bee75f6fc0e1c76238a368490f73ad22021f0a0ad73059311bbb33f5e1443255baa1fa914ab9623f36b1343c73c6b59251d9851872fa57cc28156adc9dabf35e055b2da3546745c8d0e7178628fbc54d74b8250cf9acf076867444c60020ae9fbdc5a9961329fe696403801d539da5475e6ce2eb5f72c2228e68d1efb3dc6447b41361bf65fbafcf712467bfc5eb7f807c45d5122fc875e8c29b75ae7b3b62dbf64edd212f96b8401fb1f3ebcb3bbbcbe9dce9224ac2d6716ec75a7e597f6d0abe06f84710368a715809204ca70453035fda647e0acf9fed7ed1bb82ac3b4cd1259abc7581689ee897d857866a0e04b57e30ed9cdca
Ala ma kota


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [14]:
def emsa_sign(M, emBits=2047, emlen=256, slen=32, hlen=32, H=hl.sha256):    
    mHash = int(H(bytes(M, 'utf-8')).hexdigest(), 16) 
    if emlen < hlen + slen + 2:
        raise RuntimeError('Invalid EM length')

    salt = cu.getRandomNBitInteger(slen * 8)
    M2 = bit_concat(mHash, salt, slen * 8) # not appending zeroes like before
    mHash2 = int(H(cu.long_to_bytes(M2)).hexdigest(), 16)
    PS = 0

    DB = bit_concat(0x01, salt, slen * 8)
    dbMask = mgf1(cu.long_to_bytes(mHash2), emlen-hlen-1)
    maskedDB = DB ^ dbMask
    maskedDB = nullify_first_n_bits(maskedDB, 8*emlen-emBits, 8*(emlen-hlen-1))

    EM = bit_concat(maskedDB, mHash2, hlen*8)
    EM = bit_concat(EM, 0xbc, 8)

    return EM

def emsa_verify(M, EM, emBits=2047, emlen=256, slen=32, hlen=32, H=hl.sha256):
    mHash = int(H(bytes(M, 'utf-8')).hexdigest(), 16)
    if emlen < hlen+slen+2:
        raise RuntimeError('Illegal length')

    last_byte = extract_bytes(EM, 1, 0)
    if last_byte != 0xbc:
        return False

    mHash2 = extract_bytes(EM, hlen, 1)
    maskedDB = extract_bytes(EM, emlen-hlen-1, hlen+1)

    first_bits = maskedDB & generate_bit_mask(8*emlen-emBits, emBits-8*hlen-8)
    if first_bits != 0x00:
        return False

    dbMask = mgf1(cu.long_to_bytes(mHash2), emlen-hlen-1)
    DB = maskedDB ^ dbMask
    DB &= generate_bit_mask(emBits-8*hlen-8, 0)

    zeroes = extract_bytes(DB, emlen-hlen-slen-2, slen+1)
    one = extract_bytes(DB, 1, slen)
    if zeroes != 0x00 or one != 0x01:
        return False

    salt = extract_bytes(DB, slen, 0)
    M2 = bit_concat(mHash, salt, slen*8)

    mHash3 = int(H(cu.long_to_bytes(M2)).hexdigest(), 16)
    return mHash2 == mHash3


In [15]:
M = "Ala ma podpis"

SIG = emsa_sign(M)
verified = emsa_verify(M, SIG)

print('Message  :', M)
print('Signature:', SIG)
print('Verified :', verified)

Message  : Ala ma podpis
Signature: 12692216744630081479647737000778877881152015869241942848660038988394205010342769659575677038429106863045067374916186362128710007472637491041098284445531011240289390927448310376891527230443470759818197066398139001751804850606210459211418235950204303779384763528923778385401962758992130508729176412917918227121972354262087811781163987235193940481825869674521639120631774180482735178657789850801696007897217442686214988577855256738344843689874848547757748694908566745096690610121837919661541785488468882240189090517520849792902122782099711929885693434389017042844665665292948903408161255332818433996639043080307274386620
Verified : True


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [16]:
def kem_encrypt(H = hl.sha256):
    r = cu.getRandomNBitInteger(cu.size(n))
    key_str = H(cu.long_to_bytes(r)).hexdigest()
    key = int(key_str, 16)
    ciphered_key, len = oaep_encrypt(key_str)
    return key, ciphered_key, len

def kem_decrypt(ciphered_key, len):
    key_str = oaep_decrypt(ciphered_key, len)
    key = int(key_str, 16)
    return key


In [17]:
key1, encrypted, len = kem_encrypt()
print('Generated key :', hex(key1))
print('Encrypted key :', hex(encrypted))
key2 = kem_decrypt(encrypted, len)
print('Trnsferred key:', hex(key2))
print('Keys equal?   :', key1 == key2)

Generated key : 0xf6df69a0d65d08f22e2b7ac57334bc4853dfebaf9746f50ba175833c3d2be35e
Encrypted key : 0x635cac7874f591e6cb134c532c4035121d6d0aa70e16d673a4cecc56043ac6abb0378707077b3780ab268b386bf7797faa82e83e9ff26bf48f36d94f782432af233958a300f6ac694d837c343b61a0ef009c7d99f710c5ed35a21286b41fec2ac75b27a5fbf609fa2628c2e9af943bc898340a20a7882d3a310ed237fbd4143730a3c19b165e52a5b5f43dde26e93a832cbd005c21a3d82d96b6096bdcb6aef2d6eb019dd814e3552d71bd16af629be05526cb930f2d540dfaf6e285b38462230eb852b9cec08fba36ecf08288f009de636201923774c6653d253e25710b83ea762bcc5390e8b33d9a38150953be0d970b2d3eff0a0863f0c08c22f37fce2d
Trnsferred key: 0xf6df69a0d65d08f22e2b7ac57334bc4853dfebaf9746f50ba175833c3d2be35e
Keys equal?   : True
